DOCUMENTO PARA AUTOMATIZAR EL PROCESO DE ETL , DATOS DE UBER

Librerias a importar

In [1]:
# IMPORTADO PARA WEBSCRAPPING

import re #Expresiones regulraes
from colorama import Fore
import requests
import urllib.request

In [2]:
# IMPORTAR PARA ETL Y REDUCCION DE DATOS

import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

---


CARGA DE DATOS PARA LEER -- DESPUES BORRAR

In [2]:
data = pd.read_parquet("Data/uber_final.parquet")

In [3]:
uber = data.copy()

----

# WEBSCRAPING

# Colocar el WEBCRAPIONG de leando en vez de este


Descarga todos los archivos de la pagina web

In [ ]:
#La web la que queremos scrappear
website_uber = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
#Hacemos una peticion al sitio web que seleccionamos
resultado_uber = requests.get(website_uber)
#Convertimos el resultado a texto
content_uber = resultado_uber.text
#Para hacer webscrapping tengo que encontrar en el codigo de html de la pagina un patron en comun que tengan
#las cosas que quiero extraer [\w-]*.* detecta todo lo que va despues de lo que asignamos en patron
patron_uber = r'<a\s+href="(.*?/trip-data/fhvhv_tripdata[^"]*)"'
#Queremos buscar con la expresion regular del patron dentro de content
maquinas_uber = re.findall(patron_uber, str(content_uber))
#En caso de haber duplicados debemos hacer lo siguiente:
#sin_duplicados = list(set(maquinas_repetidas))
# Debemos dejar solamente el nombre ejemplo tenemos 'data-answer="faq2023" class="faq-questions" y debe estar solo faq2023


maquinas_final_uber = []
nombre_archivos_uber = []
for i in maquinas_uber:
    if int(i[63:67]) > 2018 and int(i[63:67]) <= 2023:
        maquinas_final_uber.append(i)
        nombre_archivos_uber.append(i[48:79])
        print(i)

nombre_archivos_uber

In [ ]:
# Descarga todo en el almacenamiento local

# for a, b in zip(maquinas_final_uber, nombre_archivos_uber):
    urllib.request.urlretrieve(a, b)

# FIN WEBSCRAPING

----
----
----

# Reducción de datos al 5%

In [4]:
def reducir_archivo_uber(carpeta_origen, carpeta_destino):
    # Carpeta_origen, es la ruta de dicha carpeta
    # Carpeta_destino, es la ruta de la carpeta a ser guardado

    # Obtener la lista de archivos Parquet en la carpeta de origen
    archivos_parquet = [archivo for archivo in os.listdir(carpeta_origen) if archivo.endswith('.parquet')]

    # Iterar sobre cada archivo Parquet
    for archivo in archivos_parquet:
        ruta_parquet = os.path.join(carpeta_origen, archivo)

        # Leer el archivo Parquet y convertirlo a DataFrame
        tabla_parquet = pq.read_table(ruta_parquet)
        dataframe = tabla_parquet.to_pandas()

        # Realizar las operaciones necesarias en el DataFrame
        dataframe = dataframe.sample(frac=0.05, random_state=123).reset_index(drop=True)

        # Crear la ruta para el archivo de salida en la carpeta destino
        ruta_parquet_destino = os.path.join(carpeta_destino, archivo)

        # Guardar el DataFrame
        dataframe.to_parquet(ruta_parquet_destino, index=False)

## PARA VER!! --->  Ver tema carpetas en Google cloud!

In [5]:
carpeta_origen = 'Data/uber'
carpeta_destino = 'Data/uber/Reducido' 
reducir_archivo_uber(carpeta_origen, carpeta_destino)

# ETL DEL DATA SET DE UBERS

VER BIEN DE DONDE LO VAMOS A TOMAR! <BR>
SI LO CARGAMOS DESDE UN DRIVE, HAY QUE PONER ESE PATH

In [4]:
# Se carga para lectura el dataset brindado por TLC que posee el ID y sus respectivas localidades, distritos, zonas y zonas de servicio.

path = 'Data/taxi+_zone_lookup.csv'

taxis_zones = pd.read_csv(path)

In [5]:
# Se rellenan los nulos con el valor 'Unknown' 
taxis_zones.fillna('Unknown', inplace=True)

# Se dropea la columna LocationID
taxis_zones = taxis_zones.drop('service_zone', axis=1)


In [6]:
taxis_zones.to_parquet("Data/taxis_zones.parquet")

Sigue con el dataset de UBER

In [ ]:
# Se entiende que los registros nulos se deben a que son viajes que no fueron tomados en zona de aeropuertos, se procede a rellenarlos con valor tasa '0.00'
uber.fillna(0.00, inplace=True)

In [ ]:
# Se crea una lista con las columnas a sumar
columnas_a_sumar = ['base_passenger_fare', 'tolls', 'bcf','sales_tax','congestion_surcharge','airport_fee','driver_pay']

# Se crea una nueva columna 'total_fare' que contiene la suma de las columnas de costos
uber['total_fare'] = uber[columnas_a_sumar].sum(axis=1)

In [ ]:
# Se crea una lista con las columnas a borrar
columnasAborrar = ['base_passenger_fare', 'tolls', 'bcf','sales_tax','congestion_surcharge','airport_fee','driver_pay']

# Se dropean las columnas enlistadas
uber_sumado = uber.drop(columnasAborrar, axis=1)

In [ ]:
# Realizar la fusión (merge) basándote en las columnas LocationID, en este caso utilizando la localización de recogida
uber_merge = pd.merge(uber_sumado, taxis_zones, left_on='PULocationID', right_on='LocationID', how='inner')

In [ ]:
# Renombre de columnas
uber_merge.rename(columns={'Borough': 'PUBorough','Zone' : 'PUZone' }, inplace=True)

# Se dropea la columna LocationID
uber_merge = uber_merge.drop('LocationID', axis=1)

In [ ]:
uber_merge_parquet = uber_merge

In [ ]:
# Se realiza el merge usando las columnas de LocationID, en este caso utilizando la localización de arribo
uber_merge2 = pd.merge(uber_merge_parquet, taxis_zones, left_on='DOLocationID', right_on='LocationID', how='inner')

# Se renombran las columnas necesarias
uber_merge2.rename(columns={'Borough': 'DOBorough','Zone' : 'DOZone' }, inplace=True)

# Se dropea la columna LocationID
uber_merge2 = uber_merge2.drop('LocationID', axis=1)

In [ ]:
# Se elimina la columna que contiene la hora en que llego el conductor al lugar de recogida.
# Podria eliminarse tambien la columna de request_datetime, ya que a los objetivos del proyecto no estaría aportando algo valioso.
columnasAborrar = ['on_scene_datetime', 'request_datetime']

uber_zone = uber_merge2.drop(columnasAborrar, axis=1)

In [ ]:
# Lista de nombres de columnas que deseas convertir a tipo datetime
columnas_a_convertir = ['pickup_datetime','dropoff_datetime']

# Itera sobre cada columna y realiza la conversión a tipo datetime
for columna in columnas_a_convertir:
    uber_zone[columna] = pd.to_datetime(uber_zone[columna], errors='coerce')

In [ ]:
orden = ['pickup_datetime','PULocationID','PUBorough','PUZone','dropoff_datetime','DOLocationID','DOBorough','DOZone','trip_miles','trip_time','tips','total_fare']

# Reordena las columnas según el nuevo orden
uber_zone = uber_zone[orden]

In [ ]:
# Se renombran las columnas necesarias
uber_zone.rename(columns={'pickup_datetime': 'PUDatetime','dropoff_datetime' : 'DODatetime' }, inplace=True)

In [ ]:
uber = uber_zone

In [ ]:
# Crea dos nuevas columnas: 'fecha' y 'hora' -- para Pick Up y Drop Off
uber['PUDate'] = uber['PUDatetime'].dt.date
uber['PUTime'] = uber['PUDatetime'].dt.time

uber['DODate'] = uber['DODatetime'].dt.date
uber['DOTime'] = uber['DODatetime'].dt.time

In [ ]:
columnasAborrar = ['PUDatetime', 'DODatetime']

uber = uber.drop(columnasAborrar, axis=1)

In [ ]:
orden = ['PUDate','PUTime','PULocationID','PUBorough','PUZone','DODate','DOTime','DOLocationID','DOBorough','DOZone','trip_miles','trip_time','tips','total_fare']

# Reordena las columnas según el nuevo orden
uber = uber[orden]

# FIN DE ETL

----
----
---

# CARGA DE DATOS EN GOOGLE CLOUD

# FIN DE CARGA

---
----
---

----

PARA HACER AL FINAL EN LA TERMINAL Y GENERAR EL ARCHIVO

pip freeze | awk -F'==' '{print $1}' > requirements.txt